# 텍스트 마이닝 프로젝트 (2020.5~)
### Data Crawling
#### 휴먼지능정보공학과 류지혜, 이지민

In [5]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

#에어팟 1세대 검색 (2세대 검색하고싶으면 주소바꾸셈)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=10776906666&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25201%25EC%2584%25B8%25EB%258C%2580&NaPm=ct%3Dkb3ef8sg%7Cci%3Df7d5699aba286396ee3757965561aa228125c722%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Da9cfdf21495248f435f4ae3cbea263c694784ae1"
#xpath
shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[21]/a/span" #배터리 소모

name="에어팟 1세대" #에어팟 2세대 검색할 때 2세대로 바꾸셈
category1_="음질"
category2_="품질"
category3_="성능"
category4_="기능"
category5_="배터리 수명"
category6_="배터리 소모" 


header = {'User-Agent': ''}
d = webdriver.Chrome(executable_path=r"C:\Users\jmlee\Code\chromedriver_win32\chromedriver.exe") # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)



## 음질리뷰

In [6]:
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 

페이지 1 

1 배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 이상 없이 잘 됐어요!~연결도 잘되고, 음악 잠깐 들었는데 음질도 좋고 정말 편해요^^미리 구입해둔 케이스도 장착해서 바로 사용했어요~가격도 L.POINT랑 청구할인 받아서 직구보다 저렴하게 잘 구입했습니다.좋은 상품 감사합니다:) 5 

2 아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아이패드로 바꿔서 들을때 불편해서 블루투스 이어폰을 검색하다 주변의 구매자들 모두 하나같이 최고다라고 이야기 한 애플의 에어팟을 구매하였습니다.

할인쿠폰과 카드청구할인 혜택으로 저렴하게 구입할 수 있었고 배송도 공휴일이 포함되었음에도 2일만에 받을 수 있어 좋았습니다.

제품은 박스를 에어캡으로 감싸고 다시 큰 박스에 넣어 안전하게 배송이 되었습니다. 제품도 2018년 5월 제조 최신 제품으로 받았습니다.

받자마자 아이폰에 연결하고 사용해봤는데 . 대단하 만족합니다. 5 

3 다른 저렴한 곳도 있었습니다.

그래도 제조일자 최신에 페이백으로 다시 받는 금액 생각해보니 얼추 괜찮다고 생각해서 구매했습니다.

사용해보니 정말 편합니다. 음질도 나쁘지 않구요.
귀에 꼽고 장시간 있으면 혹시 떨어졌나 하는 생각도 들구요 가끔씩 귀 한번 만집니다 ㅋㅋㅋ
적응하면 신경 안쓸듯합니다.너무 불안하면 실리콘 줄 하나 구매해서 쓰려고 합니다 4 

4 택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사했어요
친구들 말만듣고 좋구나...생각만 했는데
막상 사용하니 너무 좋아요 어제 노래들으면서 청소 했는데 삶의 질이 달라졌다는 말이 생각 나더라구요 ㅎㅎ 너무 마음에 들어요 그리고 차에서 선이 없으니까 걸리적 거리는게 없어서 너무 편했습니다 음질도 짱짱하구요 충전도 빠르게 되는거 같고 너무 좋았습니다. 5 

5 배송 너무 빨라서 놀랐어요...금요일에 주문해서 다름주에 오겠다했는데 다음날 바로 받았어요. 귀에서 빠지면 어떡하나 

32 하이마트가 혜택이 많아서 구매했는데 배송도 빠르네요. 월 오후 주문이 화 오후에 도착했어요! 집에서 기다렸는데 택배기사님이 연락이나 초인종 없이 문앞에 놓고가셔서 하염없이 기다리긴 했습니다ㅜㅜ
우선 제품은 정말 만족하고 매일 잘 쓰고있어요. 근데 지지직 소리나 음악 채널 변경 시 틱틱 하는 잡음이 좀 있어서 며칠 더 써보다가 서비스센터에 문의해보려해요. 5 

33 음질은 아무래도 유선보다는 약간은 떨어집니다. 하지만 애플 이어팟(유선)과 비교하면 저음부가 풍부해서 어느정도 음질을 보완했다는 생각이 드네요.연결은 매우 빠르고 편리합니다. 애플기기가 2대 이상일때 전환도 매우매우 빠른 점도 좋습니다. 여기에 에어팟을 귀에 착용하면 자동으로 인식하고, 귀에서 빼면 음악이 알아서 정지되는 것도 편하네요.또, 선이 없어져서 운동할때 엄청 편하고 귀에서 잘 안빠집니다. 단순한 음향기기를 구매했다는 관점에서 보면 실망스러울 수도 있는데(비슷한 가격대에 음질 더 좋은 유선 이어폰들이 있으니), 그런걸 떠나서 생활이 꽤 편리해진다는 점이 아주 좋아요. 5 

34 계속 고민하다가 사야겠다 싶어서... 엘페이 등록하고 신한카드 청구할인으로 퀵배송으로 5천원 쓰고 받았습니다. 강남이어서 그런것인지 주문하고 30분 만에 왔어요! 완전 기분 짱이네요 ㅎㅎ 음질 좋고 간편한 연결에 짱입니다. 왜 이렇게 많이들 쓰는지 알겠네요. 5 

35 처음에 출시됐을때는 뭐야? 하는 반응으로 거들떠 보지도 않았는데
아이폰8로 바꾸면서, 이어폰 구멍이 없어져서, 알아보게 됐네요~
가격이 좀 높은편이라 몇일 고민하다가, 그래도 기존 이어폰은
불편할거같아서 구매했는데, 생각보다 귀에서 떨어질 염려도 없는거같고
통화나, 음악 음질도 좋아요!
단점은, 블루투스로 해야된다는점, ^^ 4 

36 이어폰도 분실했고 차량용 블루투스도 불편해서 겸사겸사 비싼 이걸 주문했어요 사용해보니 확실히 편하긴하더라구요 귀에도 밀착감있게 착 붙어서 쉽게 떨어질듯하진 않고. 무엇보다 이어팟을 통해 듣는 소리가 되게 선명

76 오 좋네요. 블루투스 이어폰은 처음인데 선 없는게 정말 편하고 좋네요. 음질도 생각보다 좋아요. 다 좋은데 한가지 단점은 가격이네요. 가격때문에 망설이다가 구입했는데 그래도 잘 한 것 같아요. 배송은 하루만에 바로 왔어요. 오픈형이라 귓속에 집어넣는거 싫으신 분들에게 적합합니다. 4 

77 해외배송으로 오는 다른 곳에서 주문했다가 물건 주문이 무기한 연기가 되서 11번가로 찾았는데 배송도 엄청 빠르고 제품도 2017년 제조 물건이고 안전하게 잘 왔습니다! 돈값을 하는거 같이 편리하기도 하고 음질도 좋네요!! 굿굿굿 5 

78 이어폰 줄때문에 스트레스 받아서 구입한건데 제 폰이 문제인건지...지하철에서 음악듣는데 노이즈가 너무 심합니다 전화할때도 유선이어폰과 달리 마이크가 에어팟 자체에 붙어있어서 듣는사람이 주변소음이 너무 크게들린다고 하네요 3 

79 배송짱빠름 블투이어폰 사려는데 이것도 뭔가 맘에안들고 저것도 뭔가 맘에안들고,, 이러다가 사는김에 그냥 크게 지르자 해서 에어팟구매 ! 11번가 무이자할부덕에 사는김에 애플워치도 같이 구매했음 원래 애플정품 이어폰보다 저음이 좀 빵빵해진거같고 블루투스이어폰 처음쓰는데 에어팟 진짜 짱인듯.. 5 

80 아이폰 사용자여서 이어폰잭이 없어 충전과 이어팟을 같이 못 써서 매우 불편해 큰 맘 먹고 샀네요. 이어팟 줄 풀 때마다 가위로 자르고 싶었는데 에어팟은 신세계네요 ㅎㅎㅎ 다만 성능 대비 가성비가 너무 떨어지고 음질도 그닥 좋지도 않네요… 편의성은 단연 모든 블루투스 이어폰 중 최고입니다. 돈이 넘치는 분에게 추천합니다. 3 

페이지 5 

81 일단 배송이 상당히 빨리와서 좋습니다 ㅋㅋ블루투스 이어폰도 처음인데 무선이라 어색하긴 하지만 상당히 편합니다. 음색도 이어팟보다 좀 더 선명해진 느낌입니다. 더 써봐야 알겠지만 오늘 받아서 써 본 느낌은 아주 만족스럽습니다. 애플 유저분들은 후회하지 않을 아이템이네요~ 5 

82 배송짱빠르고 드디어 줄을 안풀고 들어도 된다는게 감동이네요 ㅜㅠ 음질도 좋고 역시

224 롯데택배로 바로 보내니 배송도 빠르고 배송조회도 잘되고요 ㅎㅎ 신한카드에 엘포인트 등등 혜택도 많네요! 에어팟으로 삶의 질 올라가네요. 선없는 세상 최고에요! 쿵쿵 음질도 좋고요!! 아이폰이랑 페어링 잘되고 최고에요!! 5 

225 너무 간편하고 음질도 좋아요 최고입니다
고민끝에 비싼값을 지불하기는 했지만 막상 받고나니 만족도는 두배입니다 아깝지않아요 5 

226 배송 빨랐어요.에어팟 진짜 편하고 소리도 너무 좋아요. 5 

227 무선이라 무시했었는데 음질도좋고 역시에플 5 

228 적당한 배송과 적당한 가격에 구매한거 같습니다. 요즘 카피제품 많이 판매되는거 같던데 역시 정품이 좋은거 같네요. 음질 좋습니다. 3 

229 배송도 빠르고 음질도 좋아요!! 4 

230 배송도 빠르고 음질도 좋아요 5 

231 음질좋고 괜찮습니다. 5 

232 생각보다 음질이 좋습니다. 이어팟 비교 중저음이 좋습니다. 구매하실분 참고 하세요~~ 4 

233 무건이어폰 싼거써봤는데 페어링 끊김현상때문에 짜증납니다비싸도 품질 음질 서비스 3박자가 맞는게 좋아요 5 

234 안드로이드 폰을 사용하지만 너무 쓰고싶어서 남편꺼랑 2개 주문했어요~ 잘 되고 정품등록도 됫어요 좋네요!음질도 좋고 싸게 살수 있어서 좋아요 5 

235 진짜 너무 맘에 드네요. 음질도 좋고,제조일도 맘에 드네요.특히 배송이 빠름!오후2시쯤 구매했는데 다음날 오후 한시에 받았어요 울산인데대박! 5 

236 음악 감상이나 통화시 음질이 좋고 마음에 듭니다. 5 

237 배송도 빠르고 음질도 좋고 짱이에요!! 5 

238 음질도 좋고 포장상태도 좋네요😍 5 

239 음질 좋아요.배송도 빨라요 5 

240 갖고싶었는데이제야 갖게됐어요 ㅎㅎㅎ너무 예뻐요음질도좋구 5 

페이지 13 

241 사고 나서 다른 쇼핑몰에 더 싼 가격으로 주르륵 올라와서
좀 아쉬웠지만, 빠르게 받아 잘 쓰고 있어서 크게 불만은 없습니다. 음질 기능 모두 너무 맘에 듭니다. 진작 살 걸 그랬어요 5 

242 

353 남자친구선물로 샀어요 ㅎㅎ
음질도 좋고 무엇보다 배송이 넘 빠르네요
감사합니다 4 

354 역시 다르다는 느낌.
소리가 우선 맘에 들고,
무선의 편리함과 siri의 편리함을 만끽합니다.
가격의 값어치 한다는... 5 

355 왜 에어팟에어팟 하는지 알겠어요 솔직히 너무좋아요

귀에서 잘 빠지지도 않고 가볍고 음질도 좋고 통화도 잘되고 !!추천합니다 구매하셔도 절대 후회안하실거에요 4 

356 깔끔하게 잘 보내주셔서 감사합니다 원래 이어팟 쓰던사람이라 역시 에어팟도 좋네요 음질도 괜찮고 성능도 문제없어요~ 타 사이트보다 싸게 구매할수 있어서도 좋았습니다 5 

357 배송도 완전 빠르고 음질도 완전 좋아요ㅋㅋㅋ완전 대만족ㅋㅋㅋ 완전 좋아욬ㅋㅋㅋ 재구매 의사율도 아주 높습니다 5 

358 빠른 배송~ 선이 없어서 무척 편합니다. 가격이 비싸다는게 단점이라면 단점이죠. 핸드폰 하고 거리가3m이내 벽이 하나 있으면 소리가 끊겨요~ㅎ (블루투스의 숙명??ㅋ) 여튼 편하고 좋네요. 4 

359 가격때문에 고민 많이 했는데
타 싸이트보다 저렴하고 무엇보다 당일 주문해서 당일날 받을 수 있어서 너무 좋아요~
생각보다 음질도 괜찮고 너무 좋네요~
안잊어버리고 잘써야겠어요^^ 5 

360 설명서가 불친절한건 애플 사용자면 다 아시겠고, 음질이 생각보다 엄청 좋아서 놀랬네요. 5 

페이지 19 

361 음질이 좋긴 좋네요근데 내귀가 짝짝인가 왼쪽귀가 꽉 고정이 안되는건 왜그러지ㅠ오르쪽은 딱 좋은데요 헐 4 

362 가격이 다소 높지만 편리성 음질 괜찮네요- 운동할 때 특히 좋아요~ 5 

363 그저그래요 음질은 그냥 이어팟이고소리가 자주 끊겨서 스트레스 3 

364 완전 조아요 ㅎㅎ 음질도 좋고 이어팟이랑 착용감도 비슷해요 5 

365 음질이 너무 좋네요! 잃어버리지만 않으면 되겠습니다 5 

366 배송은 일주일넘게 걸렸어요. 음질은 좋아요 4 

367 받자마자 잘사용중!!
음질도 좋고 편하네요~ 5 

368 잘 받았습니다.. 좋아요.. 소리도 

484 편하고 음질도 좋아요 역시 가격이 젤큰 단점이지만 잘샀음여~~ 5 

485 음질은 솔직히 잘모르갯네요 이어팟보다 좋으시다고하는데 이어팟이 조금더 좋은거 같아요 그렇다고 음질이 나쁘다는건 아닙니다 4 

486 아주잘받았습니다. 음질도 너무좋고 이어폰기능도너무좋아요앞으로 또이용하고싶네요 ㅈ어말정말로 와 이렇게 좋은이어폰은 진짜또처음이네여 ㅋㅋ 다음에또 이용할께요 번청하세요 ^^화이팅이이 5 

487 애플과 호환성이 좋아 사용하기 편리하네요. 다른 블루투스 이어폰보다 음질또한 우수하고 좋아요. 강력 추천합니다.
복합할인 및 포인트 적립 그리고 롯데상품권까지 받게되면 저렴하게 잘산것 같아요. 4 

488 선없는거 진짜 좋네요. 에어팟이라서 아이폰하고 연동도 빠르고 음질도 좋네요! 5 

489 택배 진짜 빠르네요 ㅋ 가격비해 내가 원하는 음질이 아닐까봐 걱정했는데 음질 아주 마음에 듭니다 ㅎ 생각보다 스마트 하구요 강추합니다 5 

490 삼성폰 쓰고 있어서 겁정 많이 했는데, 블루투스로 연동도 잘되고 예뻐요, 음질도 좋네여~ 5 

491 에어팟 출시되고 계속 사고 싶었으나 재고가 없어서 못사고 있었어요...
이번에 재고 입고됬가는 소문을 듣자마자 구매하게 됬네요 ㅎㅎㅎ
역시 소문대로 착용감 음질 다 만족하고
애플기기와 연동이 뛰어납니다 5 

492 빠른 배송, 좋은 제품 기존부터 가지고 싶었던 제품이었는데, 정말 빠른게 배송해 주셔서 감사합니다.블루투스 연결도 잘되고, 음질도 정말 좋은 것 같아요. 계속 번창하세요.수고하세요. 5 

493 음질도좋고 엄청편하네요! 5 

494 좋아요ㅎㅎ 애플 제품인데 뭐 더 말할 필요가 있나요. 무선임에도 불구하고 음질도 좋고 착용감도 매우 좋습니다.감사합니다. 5 

495 역시 애플꺼 진짜 진짜 좋아요 블루투스라서 음질이 안좋으면 어떡하지? 했는데 역시 애플꺼 유선이랑 음질 다를게 없이 좋네요 ㅎㅎ 5 

496 좋아요 역시 소리도좋고 최고입니다. 산거 후회되지않네요~정품이고 배송도빠르게옵니다ㅎㅎ 애플좋아요

In [ ]:
def add_dataframe(type_,category,review,star):
    #n=한 카테고리당 데이터의 개수, 즉 index 수 
    
    if (len(r)>0):
        for i in range(len(r)):
            df1.loc[n]=[idx,'음식점',r[i]] 
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'음식점',0]
        n+=1
            
    if(len(c)>0):      
        for i in range(len(c)):
            df1.loc[n]=[idx,'카페',c[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'카페',0]
        n+=1
    
    if(len(l)>0):
        for i in range(len(l)):
            df1.loc[n]=[idx,'숙소',l[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'숙소',0]
        n+=1
          
            
    if(len(t)>0):
        for i in range(len(t)):
            df1.loc[n]=[idx,'교통',t[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'교통',0]
        n+=1
           
    if(len(p)>0):
        for i in range(len(p)):
            df1.loc[n]=[idx,'공원',p[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'공원',0]
        n+=1
        
    if(len(pl)>0):      
        for i in range(len(pl)):
            df1.loc[n]=[idx,'주차시설',pl[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'주차시설',0]
        n+=1
            
            
    if(len(b)>0):    
        for i in range(len(b)):
            df1.loc[n]=[idx,'술집',b[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'술집',0]
        n+=1
           
    if(len(tl)>0):    
        for i in range(len(tl)):
            df1.loc[n]=[idx,'화장실',tl[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'화장실',0]
        n+=1
            
        
    if(len(tr)>0):    
        for i in range(len(tr)):
            df1.loc[n]=[idx,'관광명소',tr[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'관광명소',0]
        n+=1
           
        
    if(len(s)>0):    
        for i in range(len(s)):
            df1.loc[n]=[idx,'쇼핑',s[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'쇼핑',0]
        n+=1
         
                
    return n #데이터의 개수 업데이트


In [ ]:
#데이터 프레임생성, 각 열의 이름 선언 (장소명,관광요소 종류, 관광요소 이름)
df1=pd.DataFrame(columns=['type','category','review','star'])

## 품질 리뷰
1. 품질, 성능, 기능, 배터리 수명, 배터리 소모 크롤링
2. [name, category, review, star] 순으로 df에 행추가해서 최종 df를 csv로 만들기 (동작구 크롤링 코드 참조)

In [ ]:
#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 

페이지 1 

1 에어팟 넘 편리하고 좋네요. 음질은 딱 이어팟 수준. 이건 좀 아쉽네요. 4 

2 음질도 좋구 만족합니다 5 

3 역시 좋아요 에어팟 정말 편하고 음질도 좋습니다. 5 

페이지 2 

4 배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 이상 없이 잘 됐어요!~연결도 잘되고, 음악 잠깐 들었는데 음질도 좋고 정말 편해요^^미리 구입해둔 케이스도 장착해서 바로 사용했어요~가격도 L.POINT랑 청구할인 받아서 직구보다 저렴하게 잘 구입했습니다.좋은 상품 감사합니다:) 5 

5 아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아이패드로 바꿔서 들을때 불편해서 블루투스 이어폰을 검색하다 주변의 구매자들 모두 하나같이 최고다라고 이야기 한 애플의 에어팟을 구매하였습니다.

할인쿠폰과 카드청구할인 혜택으로 저렴하게 구입할 수 있었고 배송도 공휴일이 포함되었음에도 2일만에 받을 수 있어 좋았습니다.

제품은 박스를 에어캡으로 감싸고 다시 큰 박스에 넣어 안전하게 배송이 되었습니다. 제품도 2018년 5월 제조 최신 제품으로 받았습니다.

받자마자 아이폰에 연결하고 사용해봤는데 . 대단하 만족합니다. 5 

6 다른 저렴한 곳도 있었습니다.

그래도 제조일자 최신에 페이백으로 다시 받는 금액 생각해보니 얼추 괜찮다고 생각해서 구매했습니다.

사용해보니 정말 편합니다. 음질도 나쁘지 않구요.
귀에 꼽고 장시간 있으면 혹시 떨어졌나 하는 생각도 들구요 가끔씩 귀 한번 만집니다 ㅋㅋㅋ
적응하면 신경 안쓸듯합니다.너무 불안하면 실리콘 줄 하나 구매해서 쓰려고 합니다 4 

7 택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사했어요
친구들 말만듣고 좋구나...생각만 했는데
막상 사용하니 너무 좋아요 어제 노래들으면서 청소 했는데 삶의 질이 달라졌다는 말이 생각 나더라구요 ㅎㅎ 너무 마음에 들어요 그리고 차에서 선이 없으니까 걸리적 거리는게 없어서 너무 편했습니

37 상품배송은 박스에 이중으로 포장되어 왔네요. 안전하게 배송된점은 좋습니다. 에어팟은 제품박스안에 본체와 충전케이블만 들어있네요. 사용설명서는 약간 부실하여 좀더 사용법을 익혀야 될 것 같습니다. 일단 블루투스연결이 편해서 좋네요. 사운드도 일반유선이어폰이랑 차이없는것 같고 통화품질도 좋은편입니다. 제품품질에 대해서는 좀더 사용해봐야 알 것 같습니다. 추가로 구매한 실리콘케이스는 본체랑 잘맞네요. 블랙으로 구매하길 잘한것 같습니다. 4 

38 에어팟 명성은 익히들어서 어디서 구매할까 하던중 쇼핑물 리뷰를 보니 여기가 제일 배송이 빠른거 같더라구요. 역시나 하루만에 도착했습니다.에어팟 박스도 손상된곳 전혀없이 아주 깨끗하게 왔구요.별5개 적극추천입니다.~~~~그리고 지금 에어팟으로 음감중인데 개인적인 감상으로는 기존 이어팟보다 저음대가 좀 더 강조되어있구요.음질은 미미하게나마 상승했다는게 절로 느껴집니다 블루투스 이어폰이라서 당연히 이어팟보다 음질이 떨어질꺼라 예상했는데 이런 대반전이 숨어있어서 가격대도 납득하구 대만족입니다. 5 

39 한달 사용 만족 합니다.위젯으로 충전기, 이어폰 배터리 표시되고배터리 출퇴근시간 풀로 들어도 넉넉하고상당히 오래 가네요막귀지만 양쪽 귀에서 악기반주 따로 들리고잡음하나 없네요귀에 걸림은 좀약한듯 하여 신경은 좀 쓰여요 5 

40 설 전에 주문해서 바로 받을 수 있을까 걱정했는데 다행히도 연휴 전에 받았네요! 역시 듣던대로 음질도 좋고 너무너무 편합니다. 써보지 못한 사람들은 느끼지 못하는 편안함! 유일한 단점은 가격... 그래도 써보면 만족할 제품입니다. 5 

41 이어폰이 한쪽은 정상 한쪽은 불량 품 받았습니다. 왼쪽은 소리가 잘들리고 베터리도 정상인데 오른쪽이 소리가 작고 베터리가 비정상적으로 빨리 떨어지네요 이상해서 베터리 어플을 깔아서 발견 그런데 15일 지났다고 반품이 안된다고 하고 서비스센터가라네요 그딴식으로 팔지 마셍 1 

42 오픈형은 자꾸 빠지고 불편해서 못끼는 사람입니다만 얘는 꼇을 때 귀에 딱 안

121 음질 최고 편의성 최고 5 

122 지금까지 이어폰 쓰고 다른 블루투스 이어폰도 써 봤지만 무엇보다 편하고 음질 좋고 완전 만족해요~~~산다산다 해놓고 자꾸 미뤄졌는데 진작에 구입할걸 그랬어요~~ 5 

123 쿠폰 이용해서 저렴하게 샀네요^^
역시 에어팟 음질도 좋고 쓰기 편해요!! 5 

페이지 8 

124 아이가 재수생 성인이 되었는데 어린이날처럼 선물 주고 싶어 구매했어요

깜짝 이벤트 해주니 너무 좋아합니다

음질 좋으니 음악도 인강도 듣기 편하다고 해요 5 

125 배송 빠르고 엄청 좋아요!!ㅎㅎㅎ
음질도 좋고 저렴하게 잘 샀어용 5 

126 진짜 신문물 접한 감동이랄까...너무 기분 좋고 음질 좋고 삶의 퀄리티가 달라졌어요ㅠㅠ 최고 5 

127 2세대 존버 실패로 그냥 얼른 구매했습니다. 진작 살걸.... 줄이 걸리는 번거로움도 없고 음질도 좋습니다. 5 

128 정품맞아요 안에 케이블선도있어서 좋구요
음질도 좋고 너무편해서 왜여태 고민했나 싶어요
하루라도 빨리사셔서 삶의질을 높이셔요 ㅠㅠ 5 

129 원래 더 저렴한 블투 이어폰 썼는데 고장나서 에어팟으류 환승했네요 헤헤 너무 좋아요 음질도 좋아 ㅠㅠ 5 

130 잘 받았습니다~~^^ 안드로이드인데 연결도 간단하고 음질도 좋네요~~~ 5 

131 편리하고 음질 모두만족합니다
딸아이선물로 너무좋아라하네요 5 

132 왜 에어팟에어팟 하는지알아버렸네요~ 배송도 정말 빠릅니다. 하루만에 왔어요 ㅋ음질도 넘 좋아요 ㅠ 많이파세요 ㅠ 5 

133 저렴한가격에 여기서 잘구매햇습니다
이쁘고 음질도 좋구요 5 

134 노트8쓰는데 연동 잘되네요~~소리도 크고 좋아요 5 

135 충전 페어링 무선 등 편의성 면에서 좋았다 아쉬운 점은 통화 음질이 아쉬웠다 4 

136 ssg도
선물용으로 구매했어요.
ssg도 선물하기가 가능하이 너무 좋아요!
방금 충전하고 사용해봤는데 음질도 그렇고 너무너무 좋다고! 하네용. 에어팟 2가 나온다곤 하지만... 넘 비싸므로 그냥 그럭저럭 잘 샀어요? 

274 호불호 있겠지만
전 귀에 딱 맞아서 고개 엄청 흔들려도 안빠지구 완전 편하구요
일반 아이폰 이어폰보다 음질이 좋은듯 (기분탓일스도)
여튼 잘샀어여 5 

275 선 없는게 이렇게 큰 차이일줄 몰랐어요 배송도 다음날 오고 무엇보다 음질도 선명하고 좋아요 편리성을 원한다면 바로 겟 4 

276 에어팟 에어팟이 역시 소리가 좋아요어마무시합니다. 사운드가 완전합니다.아주 맘에드렁요 아주 맘에 듭니다.근데 잃어버리네여ㅛ 가끔 4 

277 일단 선 없는게 젤 편한데, 음질도 나쁘지 않고 한번충전으로 출퇴근길 딱 맞네요 5 

278 배송 빠르고 오늘 받고 바로 사용. 좋아요. 음질도 좋고 만족해요 3 

279 음질도 좋고 사용 간편하게 잘 나왔네요. 배송 엄청 빨랐어요. 4 

280 음질도 생각보다 더 좋아요 잘쓰겠습니다ㅎ.ㅎ 5 

281 감사합니다 추석 연후 직후임에도 불구하고 배송이 너무 금방와서 좋습니다.잘 빠질까봐 걱정했는데 생각보다 잘 안빠지고의외로 음질이 좋아서 놀럤습니다.감사합니다 잘쓰겠습니다 5 

282 명불허전!가격은 좀 비싸지만 성능 만족합니다.음질도 좋습니다. 5 

283 배송빨라요 배송도 거의 하루만에 오고포장도 잘되어 있었어요음질도 괜찮고 만족합니당가격두 생각보다 좀 싸게 사서 만족 5 

페이지 16 

284 제품 음질이 좋아서 잘 쓰고 있어요. 착용감도 좋아서 헬스 할때도 사용해요 5 

285 왜 콩나물을 사는지 이해하겠어요~
정말 유선쓰다 무선쓰면 편리하구요 귀에도 딱 맡아서 딱히 잊어버리지 않으려고 선을 안사셔도 될거 같아요~ 음질도 정말 괜찮구요~ 5 

286 아주 잘 받았습니다! 다른곳 보다 가격도 저렴하고 질도 좋고 음질이 대단하네요 지인에게 추천해주고싶어요! 5 

287 제꺼 샀는데 남자친구가 너무 부러워해서 생일선물로 오늘 하나 더 주문했어요~ 음질도 좋고 선없으니 너무 편하네요^^ 배송도 빨라서 더 좋았어요^^ 5 

288 L포인트 쓰고 나름 16만원대로 싸게 샀어요
음질도 좋고 너무너무너무 만족 ㅠ 

403 페어링도 쉽게 잘되구 음질도 괜찮은편이어서 마음에 드네요
무엇보다 사이즈가 아담해서 너무 귀엽습니다. 4 



## 성능 리뷰

## 기능 리뷰

## 배터리 수명

## 배터리 소모